In [1]:
from pathlib import Path
from collections import Counter
from datetime import date

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.regression.linear_model import OLS
from datetime import datetime
import calendar
import math
from sklearn.metrics import mean_squared_error as mse

In [2]:
def int_sum(x):
    sum1 = 0
    for i in range(1,x+1):
        sum1+=i
    return sum1

In [3]:
DATA_PATH = Path.cwd().parent.joinpath('data', 'processed', 'three_factor_model.txt')

In [4]:
df = pd.read_csv(DATA_PATH, delimiter = "\t")

In [5]:
df["Price_tb"] = df["Price_tb"]/100

In [6]:
df.rename(columns = {"excess":"mkt_excess"}, inplace = True)

In [7]:
##making SMB
dat = {'datadate':[],'SMB value':[]}
SMB = pd.DataFrame(data = dat)
j = 0
for i in np.unique(df["datadate"]):
    temp = df[df.datadate == i][["chng","mkvaltq"]]

    ten = np.nanpercentile(temp["mkvaltq"],10)
    ninety = np.nanpercentile(temp["mkvaltq"],90)
    small = temp[temp["mkvaltq"]< ten]
    big = temp[temp["mkvaltq"]> ninety]
    small_rets = np.nanmean(small["chng"])
    big_rets = np.nanmean(big["chng"])
    factor =  small_rets - big_rets
    SMB.loc[j] = [i,factor]
    j+=1

In [8]:
##making HML
dat = {'datadate':[],'HML value':[]}
HML = pd.DataFrame(data = dat)
j = 0
for i in np.unique(df["datadate"]):
    temp = df[df.datadate == i][["tic","chng","ptb"]]

    ten = np.nanpercentile(temp["ptb"],10)
    ninety = np.nanpercentile(temp["ptb"],90)
    value = temp[temp["ptb"]< ten]
    growth = temp[temp["ptb"]> ninety]
    val_rets = np.nanmean(value["chng"])
    growth_rets = np.nanmean(growth["chng"])
    factor =  val_rets - growth_rets
    HML.loc[j] = [i,factor]
    j+=1

In [9]:
df = df.merge(SMB, how = "left", on = "datadate")

In [10]:
df = df.merge(HML, how = "left", on = "datadate")

In [11]:
date_dict = {}
for xxx in np.unique(df["datadate"])[20:25]:
    new_df = df[df["datadate"]<xxx]
    factors = pd.DataFrame(columns = ['Tic','Alpha','Beta_mkt_Exc','Beta_SMB','Beta_HML','p_MX','p_smb','p_hml','r2'])
    delta = np.zeros(shape = (len(np.unique(new_df["tic"])),len(np.unique(new_df["tic"]))))
    q = -1
    for tic in np.unique(new_df["tic"]):
        try:
            q+=1
            temp_df = new_df[new_df["tic"] == tic]
            temp_df = temp_df.reset_index(drop = True)
            temp_df = temp_df.dropna()
            Y = temp_df["chng"]-(temp_df["Price_tb"])
            X = temp_df[["mkt_excess","SMB value","HML value"]]
            w = np.ones(temp_df.shape[0])
            denom = int_sum(len(w))
            for i in range(len(w)):
                w[i] += i
            w = w/denom
            model = sm.WLS(Y,X, weights = w).fit()
            alf = 0
            for i in range(len(w)):
                no = 0
                for j in range(len(model.params)):
                    no+=X.iloc[i][j]*model.params[j]
                yes = (Y.iloc[i]-no)*w[i]
                alf+=yes

            e_i = []
            for i in range(len(w)):
                kyo = Y.iloc[i] - alf
                no = 0
                for j in range(len(model.params)):
                    no+=X.iloc[i][j]*model.params[j]
                kyo-=no
                e_i.append(kyo)

            eii = 0
            for i in range(len(w)):
                eii+=w[i]*(e_i[i]**2)
            delta[q][q] = eii

            factors.loc[q] = [tic,alf,model.params[0],model.params[1],model.params[2],model.pvalues[0],model.pvalues[1],\
                             model.pvalues[2], model.rsquared]

        except Exception as e:
            continue
    B = factors[["Beta_mkt_Exc", "Beta_SMB", "Beta_HML"]]
    ## need to rewrite f
    _, idx = np.unique(new_df["SMB value"], return_index = True)

    f = new_df[["datadate","mkt_excess","SMB value","HML value"]].iloc[np.sort(idx)].reset_index(drop = True)
    w = np.zeros(len(f))
    denom = int_sum(len(w))
    for j in range(len(w)):
        w[j] += j
    w = w/denom
    F = np.zeros(shape = (3,3))
    for t in range(len(f)):


        temp = np.zeros(3)
        for s in range(len(f)):
            temp = temp + w[s]*f.iloc[s][["mkt_excess","SMB value","HML value"]]

        term1 = f.iloc[t][["mkt_excess","SMB value","HML value"]]-temp

        mat1 = np.dot(np.reshape(np.array(term1),(3,1)),np.transpose(np.reshape(np.array(term1),(3,1))))
        mat1 = mat1*w[t]
        F = F + mat1
    V = np.dot(np.dot(np.array(B),F),np.transpose(np.array(B))) + delta
    qqqq = {}
    qqqq["factor_loadings"] = factors
    qqqq["V"] = V
    qqqq["factors"] = f
    date_dict[xxx] = qqqq

/Users/siddharthkantamneni/anaconda3/lib/python3.7/site-packages/statsmodels/regression/linear_model.py:1510: RuntimeWarning: divide by zero encountered in double_scalars
  return np.dot(wresid, wresid) / self.df_resid
/Users/siddharthkantamneni/anaconda3/lib/python3.7/site-packages/statsmodels/regression/linear_model.py:1543: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
/Users/siddharthkantamneni/anaconda3/lib/python3.7/site-packages/statsmodels/regression/linear_model.py:1510: RuntimeWarning: invalid value encountered in double_scalars
  return np.dot(wresid, wresid) / self.df_resid
/Users/siddharthkantamneni/anaconda3/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/Users/siddharthkantamneni/anaconda3/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in les